Import the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

Data preprocessing

In [3]:
df = pd.read_csv(r'C:\Users\Stelios_Ntanavaras\Documents\Python Scripts\Machine-Learning-A-Z-AI-Python-R-ChatGPT-Prize-2024-\Machine Learning\Deep Learning\Churn_Modelling.csv')
df.head(100)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,15699461,Fiorentini,515,Spain,Male,35,10,176273.95,1,0,1,121277.78,0
96,97,15738721,Graham,773,Spain,Male,41,9,102827.44,1,0,1,64595.25,0
97,98,15693683,Yuille,814,Germany,Male,29,8,97086.40,2,1,1,197276.13,0
98,99,15604348,Allard,710,Spain,Male,22,8,0.00,2,0,0,99645.04,0


In [4]:
X = df.iloc[:, 3:-1].values # We exclude the 'Customer ID', and the 'Surname' because they do not help to predict the dependent variable 'Exited' 
y = df.iloc[:, -1].values

print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [5]:
# Label categorical data

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) # 'Gender' column


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough') # 'remainder' arg is to keep the columns that the transformation is not applied 
X = np.array(ct.fit_transform(X)) # 'Geography' column

print(X)


[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [6]:
# Split the dataset into Training and Testing set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
# Feature Scaling

# ABSOLUTELY IMPORTANT FOR DEEP LEARNING
# APPLY IT TO ALL FEATURES

from sklearn.preprocessing import StandardScaler

# First Technique: STANDARDIZATION (almost all the values inside the columns will be mapped in a range of [-3,3])
# stand = x - mean(x) / std(x)

sc = StandardScaler()
X_train = sc.fit_transform(X_train) # '.fit' will find the mean and std of all values, and '.transform' will apply the formula
X_test = sc.fit_transform(X_test)

print(X_train)
print(X_test)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]
[[-0.95692675  1.62776996 -0.57427105 ...  0.66011376  0.97628121
   1.62185911]
 [ 1.04501206 -0.61433742 -0.57427105 ...  0.66011376 -1.02429504
   0.504204  ]
 [-0.95692675 -0.61433742  1.74133801 ...  0.66011376  0.97628121
  -0.41865644]
 ...
 [-0.95692675 -0.61433742  1.74133801 ...  0.66011376 -1.02429504
   0.72775202]
 [-0.95692675  1.62776996 -0.57427105 ...  0.66011376  0.97628121
  -1.54162886]
 [-0.95692675  1.62776996 -0.57427105 ...  0.66011376 -1.02429504
   1.62356528]]


Building the ANN (with Tensorflow)

In [8]:
# Initialize the model
ann = tf.keras.models.Sequential()

# Add the input layer and the 1st hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) # 'units = 6' is the default option

# Add the 2nd hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

# Add the output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

Training the ANN

In [9]:
# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training
ann.fit(X_train, y_train, batch_size = 32, epochs = 100) # default options

Epoch 1/100
250/250 [==============================] - 5s 6ms/step - loss: 0.5773 - accuracy: 0.7390
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4790 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4491 - accuracy: 0.7983
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4337 - accuracy: 0.8054
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4211 - accuracy: 0.8096
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4080 - accuracy: 0.8102
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3944 - accuracy: 0.8183
Epoch 8/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3798 - accuracy: 0.8344
Epoch 9/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3671 - accuracy: 0.8511
Epoch 10/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3566 - accu

Evaluation

In [10]:
# Predicting the Test set Results
y_pred = ann.predict(X_test) # what I predict
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)) # y_test are the true values

# left: predicted values,  right: true values


# Confusion Matrix 
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred) 

print('The confusion matrix is:', cm)
print('The accuracy score is:', accuracy_score(y_test, y_pred))

63/63 [==============================] - 2s 9ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
The confusion matrix is: [[1543   52]
 [ 218  187]]
The accuracy score is: 0.865
